### **6.3 - [Practica] Grouping your tasks with SubDAGs and Deadlocks**

Ya hemos visto lo que es un SubDag así que es hora de poner en práctica lo aprendido. Para ello, abre tu editor de código y comprueba que te encuentras en la carpeta airflow-materials/airflow-section-6. Ahora, en la carpeta mnt/airflow/dags, abre el DAG test_subdag.py.  Este DAG corresponde al ejemplo mostrado en el vídeo anterior.  

<center><img src="https://i.postimg.cc/Bvx4DF7K/a928.png"></center>
<center><img src="https://i.postimg.cc/MHrSxmHh/a929.png"></center>

Tenemos tres tareas llamadas start, check y final con dos SubDags.  El primero llamado subdag-1 entre start y check. Y el segundo llamado subdag-2 entre check y final.  Si tomamos la tarea subdag-1 como ejemplo, se utiliza un SubDagOperator con el parámetro "subdag" tomando el "factory method" que crea el subdag.  Esta función toma tres parámetros.  El primero es el nombre del DAG padre que es "test_subdag" definido por la variable "DAG_NAME".  El segundo parámetro es el nombre del subdag que es "subdag-1". Y el último parámetro corresponde a los parámetros por defecto establecidos para el DAG padre que aplicamos también al subdag.  ¿Por qué?  Porque si recuerdas, tenemos que mantener una especie de coherencia entre el padre y el hijo con el start_date y scheduling_interval para evitar comportamientos inesperados.  Entonces, ¿de dónde viene la función factory_method?  Si te fijas en la parte superior del archivo, he hecho una importación desde el archivo subdag. Así que, si abres este archivo desde la carpeta subdags, obtienes el factory method.   

<center><img src="https://i.postimg.cc/qMST5w9K/a930.png"></center>

Tenemos los tres parámetros utilizados para instanciar un objeto DAG como se muestra aquí. Por convención, el dag_id de un subdag está compuesto primero por el DAG id del padre y luego el DAG id del subdag separados por un punto, como puedes ver aquí. Luego, el bucle aquí crea 5 tareas en el subdag. En resumen, necesitas crear un factory method en un archivo externo encargado de devolver un objeto DAG. Entonces, este objeto DAG es tu SubDAG que se adjuntará a tu DAG padre con el operador SubDagOperator.  Antes de seguir adelante, si vuelves al archivo "test_subdag", establecí el Sequential Executor para cada SubDagOperator. Ese es el executor por defecto. Si quitas este parámetro obtendrás el mismo comportamiento. Vamos a cambiarlo en un minuto, pero primero vamos a ejecutar este DAG desde la interfaz de usuario de Airflow. En primer lugar, abra su terminal y compruebe que se encuentra en la carpeta airflow-materials/airflow-section-6. Si es así, escribe el siguiente comando "docker-compose -f docker-compose-CeleryExecutor.yml up -d".  Enter.  Bien podemos comprobar que Airflow se está ejecutando, escribiendo "docker ps". Perfecto. Abre tu navegador web, escribe localhost:8080, Enter. Activa el toggle del DAG "test_subdag" y refresca la página.  Ok, el DAG está en ejecución, espera a que termine el DAGRun.  Muy bien, si haces clic en el DAG, luego en "Graph View", puedes ver dos tareas subdag_1 y subdag_2 con bordes en negrita. 

<center><img src="https://i.postimg.cc/QxHDmwns/a931.png"></center>

Haz click en subdag_1, y "zoom into Sub Dag". 

<center><img src="https://i.postimg.cc/nz2xSJJV/a932.png"></center>
<center><img src="https://i.postimg.cc/y6fBFsjv/a933.png"></center>

Como se puede ver por el nombre aquí, estamos en el subdag "subdag_1". Desde la Graph view, tenemos las cinco tareas creadas a partir del factory method que vimos anteriormente.

<center><img src="https://i.postimg.cc/SRKhwvS6/a934.png"></center>

Desde la Gantt view, podemos observar que las tareas fueron ejecutadas secuencialmente.  

<center><img src="https://i.postimg.cc/qR40G7WZ/a935.png"></center>

Dado que, por defecto, el SubDagOperator se establece con el Sequential Executor, este es el comportamiento normal, incluso si Airflow se está ejecutando con el Celery Executor.  Entonces, ¿cómo podemos ejecutar múltiples tareas en paralelo incluso en SubDags?  Bueno, para ello, vuelve al archivo "test_dag.py" y cambia de Sequential Executor a Celery Executor para ambos subdags.  Guarda el archivo. 

<center><img src="https://i.postimg.cc/Sx9hF8hv/a936.png"></center>

De vuelta al navegador, haz clic en "DAGs". Compruebe que las modificaciones realizadas se aplican haciendo clic en el DAG, y luego en "Code". Ok, ya tenemos el Celery Executor. Actualice el DAG haciendo clic aquí para asegurarse de que las últimas actualizaciones se tienen en cuenta.  Ok.  Ahora hacemos click en "Trigger DAG" justo aquí. "Ok", y un nuevo DAGRun se dispara. Actualiza la página hasta que el DAGRun haya terminado.  Ahora que está hecho, haga clic en "Graph View", "subdag_1", "Zoom into Sub DAG", "Gantt view", y como se puede ver las tareas se ejecutaron en paralelo.  

<center><img src="https://i.postimg.cc/wBdd6vxT/a937.png"></center>

Muy bien, me gustaría indicarte algo.  Si vuelves al DAG padre pulsando aquí, puedes observar en la tree view que un SubDag se representa como una tarea en su DAG padre y no como un grafo de tareas.  

<center><img src="https://i.postimg.cc/CKHpdgdC/a938.png"></center>

Esta es la razón por la que pueden producirse bloqueos. Un subdag es una abstracción de tareas, pero aún así se comporta como una tarea.  Déjame darte un ejemplo para que quede más claro.  Haga clic en "Dags". Pausa el DAG "test_subdag" apagando el toggle y enciende el toggle del DAG "deadlock_subdag".  Ahora si haces click en el DAG y empiezas a refrescar la página incluso después de 15 minutos, los subdags seguirán en estado de ejecución (running state). 

<center><img src="https://i.postimg.cc/xCfDz4bw/a939.png"></center>

Entonces, ¿por qué los subdags están atascados en estado de ejecución?  Bueno, con la configuración actual de Airflow, hasta 4 tareas se pueden ejecutar en paralelo ya que el parámetro parallelism se establece en 4. Si comprueba la "Graph View" se puede ver que tenemos 4 SubDags que se pueden ejecutar en paralelo donde cada uno contiene 5 tareas que también se pueden ejecutar en paralelo ya que los SubDagOperators se establecen con el CeleryExecutor. 

<center><img src="https://i.postimg.cc/LXxdB3tn/a940.png"></center>

Podemos comprobarlo rápidamente volviendo al editor de código y abriendo el archivo "deadlock_dag".  Aquí, tenemos el CeleryExecutor establecido. 

<center><img src="https://i.postimg.cc/vTTJPxhX/a941.png"></center>
<center><img src="https://i.postimg.cc/tJcL8Rmb/a942.png"></center>

De vuelta a la interfaz de usuario.  Si recuerdas, dije que un subdag se representa como una tarea y, por lo tanto, todas las tareas en este subdag deben tener éxito con el fin de obtener el subdag marcado como éxito (succeed) también.  

<center><img src="https://i.postimg.cc/c15yvT1z/a943.png"></center>

Esto significa que un subdag ocupa un worker slot hasta que todas las tareas hijas se ejecutan.  Dado que tenemos 4 SubDags y hasta 4 worker slots están disponibles para ser tomados en paralelo, los Subdags están todos esperando para terminar de ejecutar sus tareas, pero ya no hay ningún slot para ejecutarlas y por lo tanto todo se atasca.  Así es como se llega a un punto muerto o bloqueo (deadlock).  

<center><img src="https://i.postimg.cc/yYgqc9k8/a944.png"></center>

Ahora la pregunta es ¿cómo se puede evitar esto?  Bueno, una forma de resolver este problema es añadiendo una cola de los SubDagOperators para que se ejecuten en esta cola dedicada y no tomen el lugar de las tareas hijas.  Luego, creas un nuevo worker node y le asignas esa cola.  Te dejo esto como un ejercicio ya que deberías ser capaz de hacerlo ahora que has visto cómo crear colas, pools y workers en la sección 5.  **`Así que esta es una solución fácil, pero en mi opinión, te recomiendo que te mantengas alejado de los SubDags o si quieres usarlos, quédate con el Sequential Executor.  Esto te mantendrá alejado de mucho dolor`**.  Bien, vuelve a tu terminal, y detén los contenedores docker escribiendo "docker-compose -f docker-compose-CeleryExecutor.yml down".  Enter. Perfecto.  Espero que te haya gustado lo que has aprendido sobre los deadlocks así que ten cuidado si estás tratando con SubDags en producción. Por cierto, si quieres el video sobre la solución para arreglar los deadlocks, por favor házmelo saber en la sección Q/A del curso.  Nos vemos en el próximo vídeo.  